In [16]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.special import ellipe
from scipy.special import ellipk

#parameters
G = 4.300e-6                                        #gravitational constant (kpc/solar mass*(km/s)^2)
rho = 1e5                                           #madeup parameter until we figure out the partial fraction of rho(u,xi)
h = 5.7                                             #radial scale-length (kpc)
z0 = 0.83                                           #half-thickness (kpc)
R = 19.3                                            #19.3 (kpc)
d = 1                                               #cut-off length upper limits(kpc)
mu0 = 1e9                                           #madeup parameter: central surface density

#definitions
x = lambda r,u,xi: ((r**2)+(u**2)+(xi**2))/(2*r*u)
p = lambda x: x-np.sqrt((x**2)-1)
r = np.linspace(0, 20, 500)
rho00 = mu0/(2*z0)

print(p(x(1,1,0)))

1.0


In [2]:
#density piecewise function
from scipy import optimize

def rho0(r, R, h, d):
    condlist = [r <= R, (r > R) & (r <= (R+d)), r > (R+d)]
    funclist = [lambda r: rho00*np.exp(-r/h), lambda r: rho*(1-(r-R)/d), lambda r: 0]
    return np.piecewise(r, condlist, funclist)

#p , e = optimize.curve_fit(rho0, r, y)

#rd = np.linspace(1, 30, 1000)
#plt.plot(r, y, "o")
#plt.plot(rd, rho0(rd, *p))

In [18]:
#need: partial derivative of rho(u,xi)
def durho0(r, R, h, d):
    condlist = [r <= R, (r > R) & (r <= (R+d)), r > (R+d)]
    funclist = [lambda r: -(1/h)*rho00*np.exp(-r/h), lambda r: -(1/d)*rho, lambda r: 0]
    return np.piecewise(r, condlist, funclist)

#complete elliptic integral
K = lambda r,u,xi: ellipk(p(x(r,u,xi))) - ellipe(p(x(r,u,xi)))

print(K(1,1,1))

0.35734399698214614


In [19]:
#disk density distribution
rho_rz = lambda r,z: rho0(r,R,h,d)*(np.power(np.cosh(z/z0), (-2)))
drho_rz = lambda r,z: durho0(r,R,h,d)*(np.power(np.cosh(z/z0), (-2)))

print(rho_rz(0,0))

602409638.0


In [14]:
#inner function
f = lambda r,u,xi: ((2*np.sqrt(u))/(np.pi*np.sqrt(r*p(x(r,u,xi)))))*K(r,u,xi)*drho_rz(u,xi)

print(f(1,1,1))

-9876768.647189368


In [ ]:
#plot f


In [ ]:
#integrate inner function
g = lambda xi: quad(f, 1, np.inf, args=(xi,))[0]

In [7]:
#integrate outer function
j = lambda u: quad(g, 1, np.inf, args=(g,))[0]

In [8]:
#radial force function
F = 4.0*np.pi*G*j

TypeError: unsupported operand type(s) for *: 'float' and 'function'

In [ ]:
#graph
